In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Imports - our files
import utils
import models

# Global definitions - data
DATA_FN = 'data/crowdflower_data.csv'
LABEL_NAMES = ["happiness", "worry", "neutral", "sadness"]

# Global definitions - architecture
EMBEDDING_DIM = 100  # We will use pretrained 100-dimensional GloVe
BATCH_SIZE = 128
NUM_CLASSES = 4
USE_CUDA = torch.cuda.is_available()  # CUDA will be available if you are using the GPU image for this homework


In [2]:
train, dev, test = utils.get_data(DATA_FN)
# 接下来vectorize_data 不同


Random samples from train set
       sentiment                                            content
20425  happiness    Listening to Miley Cyrus Breakout CD ! love it!
4330       worry            Really nice outside...I'm stuck at work
2321       worry  @christineduggan we keep getting pet days  but...
5138     sadness  Dang my own LDBF didnt give me a Follow Friday...
24463      worry  Please remember you need a helmet for the bike...
18368  happiness  @Sierrasnowboard under a week delivery to OZ, ...
8251     sadness  holy wow, I think I could've slept all day! So...
10756      worry                       Ok the thunder is scaring me
18269    neutral  @thatissodope Errr, yes? ) Sa enrollment sasab...
5307     sadness  @cameronmarion me too  no one even liked my an...





In [3]:
np.shape(train)

(21976, 2)

In [5]:
train.iloc[:1]

,sentiment,content
1513,worry,I caaaaan't sleep.....it's 3.30am! wahhhh... i...
